<img src="https://drive.google.com/uc?id=1-hPP-XPm9_5M3orUgmompcVleQ5xvPST" style="Width:1000px">

# Predicting next month's temperatures

In this exercise, you will use a modified version of the world's city temperature. The main goal of this exercise is to introduce you to some more advanced feature engineering techniques, notably `Feature Cross`, `Cyclic Data Encoding`, and `Feature Hashing`. Settle down: this exercise will push your coding skills a bit more!

# Reading the data

I have already split the data for you into a `train_set` and a `test_set` in the cell below. Note that because the data is a `Time Series` I have split it in a way that all of our `train_set` lies between 2003 and 2011m whereas the `test_set` represents the years 2012, and 2013 (we will discuss next week in greater details why it is advisable to do this - for now, just go with the flow!).

Once you have executed the cell below, look at the `train_set` to see what the features we are going to work with look like.

In [ ]:
from nbta.utils import download_data
download_data(id='1pm9Y1uOGgRsfnhGHZUpsdSXmF3wgtXK2')

In [ ]:
# Run this cell first in order to be able to see changes in your "custom_transformers.py" file without needing to restart your kernel
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import numpy as np

train_set = pd.read_csv('raw_data/train_temperatures.csv')
test_set = pd.read_csv('raw_data/test_temperatures.csv')

# Reducing the size of the data

Some of you, especially running on WSL, will struggle with the size of this data so we will only take 25% of our train and test splits. But if you are curious, you can play with the entire dataset later (assuming you have enough memory).

In [11]:
train_set = train_set.sample(frac=0.25, random_state=42)
test_set = test_set.sample(frac=0.25, random_state=42)

In [ ]:
# ADD YOUR CODE HERE -- You can create new markdown and code cells
                    
                    
                    

## Our objective

We will try to predict the `AverageTemperature` based on the different features you can see in the `DataFrame`. All of them should be relatively easy to understand intuitively. So, let's look at a visual of our data to get a sense of what we are dealing with.

# Visualization

To better understand our challenge, let's visualize it for a given month, of a given year. Do the following:

1. Create a new `DataFrame` called `may_2004` that will only contain the data from your `train_set` that corresponds to the year 2004 and the month of May.
2. Create a new feature in your `may_2004` dataframe, and call it `temp_diff`. This is equal to the `TempPreviousMonth` minus the `AverageTemperature`. In other words, it tells us what the temperature difference between the month of April 2004 and the month of May 2004 looks like. This is what we will try to predict for all months of the year based on the entire dataset.
3. Now it is time to plot our data! Create a new figure with axes, I suggest using a `figsize=(12,8)` or something similar to be able to view the data well. Scatter `may_2004.Longitude` against `may_2004.Latitude`, and set the color of each point to be controled by `may_2004.temp_diff` (look at the parameter `c` of the matplotlib `scatter()` function. I also recommend that you use the `coolwarm_r` color map (`cmap` parameter) for easy visualization

What can you see, and does it make sense?

In [ ]:
# ADD YOUR CODE HERE -- You can create new markdown and code cells
                    
                    
                    

<h2>💡 Conclusions</h2>
<br>
If you followed the instructions above, you should now see the following:<br>
    <li>Each city plotted gives the rough contour of the continents.</li>
    <li>In the Southern Hemisphere, the dots are mostly <span style="color:blue">blue</span>, indicating negative values of temperature difference (i.e. May is cooler than April, as we are moving towards the Austral Winter.</li>
    <li>In the Northern Hemisphere, the dots are <span style="color:red">red</span>: this is Spring warming</li>
    <li>In the Equatorial region, the dots are <span style="color:gray">gray</span>: this is because temperatures are fairly equables in the tropic (i.e. don't change much)</li>
<br>
    So what do we learn here? That the change of temperature will depend on <strong>the season</strong> and the <strong>geographical location</strong>. Luckily, we have information on that:
    <li><strong>Geographic location</strong>: we have the latitude and longitude of the cities, and also the city names!</li>
    <li><strong>Season</strong>: we have information about the month when the data was measured</li>
    
We will look at different ways to incorporate this information in this exercise.

### ☑️ Test your code

In [ ]:
from nbresult import ChallengeResult

result = ChallengeResult('visualization',
                         df = may_2004
)

result.write()
print(result.check())

# Creating features and target

Time for us to create our features (`X_train` and `X_test`) and our targets (`y_train` and `y_test`). Use the `AverageTemperature` as your targets for both the `test_set` and the `train_set`. And for the `X_train` and `X_test` drop the `AverageTemperature` and keep the others: 

In [ ]:
# ADD YOUR CODE HERE -- You can create new markdown and code cells
                    
                    
                    

# Baseline Score

We need to have an idea of what score we need to beat to be able to claim that we have a model that is performing better than a random guess. In our case, a baseline model to beat would be using the temperature of the previous month as our prediction.

Do the following:
1. Calculate the `root mean squared error` between `X_test.TempPreviousMonth` and `y_test`. 
2. Save this value into a new variable named `baseline_score`

In [ ]:
# ADD YOUR CODE HERE -- You can create new markdown and code cells
                    
                    
                    

### ☑️ Test your code

In [ ]:
from nbresult import ChallengeResult

result = ChallengeResult('baseline_score',
                         score = baseline_score
)

result.write()
print(result.check())

# Training a Linear model based on numerical coordinates

In our first attempt to model the temperature a month ahead, we will use the `numerical coordinates` (i.e. `Latitude` and `Longitude`) only. We won't beat our `baseline_score` just yet: we will work towards that throughout this workbook!

Do the following:
1. Import `set_config` and use `set_config(display='diagram')` for easy visualization of your pipelines 
2. Create  a `ColumnTransformer` that contains a `StandardScaler`, and make sure to only select the columns `Latitude` and `Longitude` for this step. I suggest you call this transformer `preproc`.
3. Combine your `preproc` and a `LinearRegression` model into a new pipeline: call it `coord_model`
4. Train `coord_model` on `X_train` and `y_train`
5. Calculate the `root mean squared error (RMSE)` for `coord_model` using your `X_test`. Save this value in a variable named `coord_score` (make sure to look at it!)

In [ ]:
# ADD YOUR CODE HERE -- You can create new markdown and code cells
                    
                    
                    

<details><summary><strong style="color:darkgreen">Are you happy with this score?</strong></summary><br>
    This is obviously lower than our <code>baseline_score</code> but is to be expected: coordinate alone cannot be a better predictor than the previous month temperature. <br>
    More importantly, giving <strong style="color:blue">numerical values</strong> is not the best way to encode geospatial data: intruitively, <strong>BOTH</strong> the <code>Latitude</code> and the <code>Longitude</code> should be given together (not in separate columns) to pinpoint a location on the planet. In the next section, we will discuss how this information can be encoded for our machine learning algorithm.

### ☑️ Test your code

In [ ]:
from nbresult import ChallengeResult

result = ChallengeResult('num_coordinates',
                         score = coord_score
)

result.write()
print(result.check())

# Introducing `Feature Crosses`

This brings us to a very important concept: that of a `feature crosses`. Simply put, a feature cross is a synthetic feature that is the combination of two or more features together: independently, none of these features may be very strong predictors. But when associated, they become powerful! You can <a hread="https://developers.google.com/machine-learning/crash-course/feature-crosses/video-lecture#:~:text=A%20feature%20cross%20is%20a,an%20understanding%20of%20feature%20crosses.">watch this brief video from Google</a> about `feature crosses` if you want to know more, or <a hreaf="https://www.linkedin.com/pulse/why-feature-crosses-still-important-machine-learning-rakesh-sharma/">read this article</a>.

In our case, the coordinates of the city give us a clear opportunity to practice feature crossing: we have both the `Latitude` and the `Longitude` of the cities in separate columns. However, a location is a combination (`feature cross`!) of both `latitude` and `longitude`. Simply creating a new column that contains a string in the form of `{Latitude}_{Longitude}` would not be enough though: we need to encode the entire space of possible parameters to create a high-dimentional feature space. Then we can represent each combination of `Latitude` and `Longitude` as a unique vector.

To achieve this, you will need to write three functions: `train_encoder`, `encode_features`, and `create_feature_cross`. Let's walk through them one by one.

### `train_encoder`

The purpose of this function is to train a `OneHotEncoder` on all possible combinations of `Latitude` and `Longitude` rounded to the nearest full degree. The function will return a trained `OneHotEncoder`. Here are a few useful tips when writing this function:

1. Remember that you can cast a `float` to an `int` to round the values: I suggest you use this approach here
2. You might want to generate two lists: one of all possible `latitudes` (from `-90` to `+90`) and one of all possible `longitudes` (from `-180` to `+180`). **Remember:** We are trying to encode each full degree here, i.e. `int`s. The `np.arrange` function might be your best friend here!
3. You probably want to create a list that will contain all of your possible combinations of features (let's call it `corpus`, but you can choose another name)
4. You might want to loop through all `latitudes`, and then for each `latitude` value combine it with all possible values of `longitude`. Then append this combination as a string (for instance, `"34_-123"` for 34N and 123W) to your list
5. To save you a lot of grief in your next function, I recommend you turn your `corpus` list into a `pd.DataFrame` with a column name (for instance, `coordinates`)
6. Now, create a `OneHotEncoder` (important: do **NOT** set the parameter `sparse` to `False`. We definitely want sparsity for speed of training and memory usage), and `fit` your encoder on the `DataFrame` version of your corpus.
7. Return your fitted `OneHotEncoder`.

Check your code to see if it works!

In [ ]:
# ADD YOUR CODE HERE -- You can create new markdown and code cells
                    
                    
                    

### ☑️ Test your code

In [ ]:
from nbresult import ChallengeResult

result = ChallengeResult('encoder',
                         encoder = train_encoder()
)

result.write()
print(result.check())

### `encode_features(row)`

The goal of this function is to take as input a single `row` of one of your `DataFrame` (for instance, the `X_train` or `X_test`, and return a `string` of your `row.Latitude` and `row.Longitude` in the same format as you trained your `OneHotEncoder`. For instance, if you followed my example above, the function should return a string `"23_-5"` for the values of `Latitude=23.6` and `Longitude=-5.06`. If you encoded your data differently in your `train_encoder` function, then follow what you did previously.

In [ ]:
# ADD YOUR CODE HERE -- You can create new markdown and code cells
                    
                    
                    

### ☑️ Test your code

In [ ]:
from nbresult import ChallengeResult

result = ChallengeResult('encode_features',
                         encoded_string = encode_features(pd.Series([23.6, -5.06], index=['Latitude', 'Longitude']))
)

result.write()
print(result.check())

### create_feature_cross(df)

This function will take as input a `DataFrame` (I call it `df`), and return a sparse matrix of your coordinates encoded. Don't worry, you already did most of the hard work! Here is what you need to do in this function:

1. Create an encoder by using the `train_encoder` function you wrote above. You will use this encoder to encode your data.
2. Create a vector of strings that represents your coordinates. Because you already have written a fonction to turn coordinates into strings, it is easy. I suggest you use the `apply()` method on `df`, and use a `lambda function` to turn each row (think of setting `axis=1` in `apply()) into a string. Save the return value of this function (a vector) in a variable (I call it `vector`).
3. Turn your vector into a `DataFrame` with one column named `coordinates`
4. Use your `OneHotEncoder` to `transform` your `DataFrame`, and return the transformed value from your function

You are done!

In [ ]:
# ADD YOUR CODE HERE -- You can create new markdown and code cells
                    
                    
                    

### ☑️ Test your code

In [ ]:
from nbresult import ChallengeResult

test_df = pd.DataFrame(data=[[23.6, -176.4], [-5.06, 90.0]], columns = ['Latitude', 'Longitude'])
result = ChallengeResult('create_feature_cross',
                         sparse_matrix = create_feature_cross(test_df)
)

result.write()
print(result.check())

## Modelling with our `Feature Hash`

Now comes the exciting time: we shall see if our `Feature Hash` is more predictive than simply using he numrical coordinaes on their own. Do the following:

1. You can create  `X_train_hashed` and `X_test_hashed` by applying the `create_feature_cross()` function to `X_train` and `X_test`, respectively.
2. Use the `X_train_hashed` to train a new model (`fc_model`) that contains only a `LinearRegression`
3. Calculate the new `root mean squared error` and save it into a variable named `fc_score`

Do we beat our old `coord_score`?

In [ ]:
# ADD YOUR CODE HERE -- You can create new markdown and code cells
                    
                    
                    

### ☑️ Test your code

In [ ]:
from nbresult import ChallengeResult

result = ChallengeResult('fc_score',
                         score = fc_score
)

result.write()
print(result.check())

# `CoordinatesFeatureCross` Transformer

Write a class implementing a transformer that is similar to the code you produced above it aims at encoding a `Feature Cross` of your latitude and longitude coordinates (do this in the `custom_transformers.py` file using `VSCode`). It returns a sparse matrix of the encoded features.

In [ ]:
from custom_transformers import CoordinatesFeatureCross

cfc = CoordinatesFeatureCross().fit(X_train, y_train)
feature_cross = cfc.transform(X_test)

### ☑️ Test your code

In [ ]:
from nbresult import ChallengeResult

result = ChallengeResult('feature_cross_transformer',
                         sparse_matrix = feature_cross
)

result.write()
print(result.check())

# Modelling with cities name

We will now take advantage of the `City` feature. Because we know that location matters for our prediction, we can surmise that adding the city name would matter too. But before we proceed with that, let's think of whether the city name in itself is a sufficiently unique feature. Look at the `unique` values in `X_train` for `City==London`. What do you conclude?

In [ ]:
# ADD YOUR CODE HERE -- You can create new markdown and code cells
                    
                    
                    

### ☝️ `City` is not unique

Hopefully the above should have convinced you that the `City` feature is not unique (the same city name can appear in different countries). To work around this, create a new feature called `"City_Country"` that is a string that pairs the city and country with a `_` character. For instance, replace `London` by `London_United Kingdom`. Do this for both the `X_train` and the `X_test`. Do not  drop the `Country` feature and `City` yet: we will need those later!

In [ ]:
# ADD YOUR CODE HERE -- You can create new markdown and code cells
                    
                    
                    

### ☑️ Test your code

In [ ]:
from nbresult import ChallengeResult

result = ChallengeResult('city_country',
                         X_train = X_train,
                         first_city=X_train.City_Country[0]
)

result.write()
print(result.check())

## Modelling with Cities [attempt #1]

Now that you have replaced the `City` feature by a more unique feature name (the `City_Country` feature), it is time to see how we can model it. Do the following:
1. Create a numerical pipeline that contains a `StandardScaler`, and a categorical pipeline that contains a `OneHotEncoder`. We will attempt to use `OneHotEncoding` for our `City_Country` feature.
2. Combine both pipelines into a `ColumnsTransformer`: select `['TempPreviousMonth','Latitude', 'Longitude']` as features for the numerical pipeline, and `City` for the categorical pipeline. Set `remainder='drop'` for the `ColumnTransformer`
3. Combine your `ColumnTransformer` with a `LinearRegression()` into a new pipeline called `City_model`, and `fit` this model to `X_train` and `y_train`.

You should now have a fully fitted model.

In [ ]:
# ADD YOUR CODE HERE -- You can create new markdown and code cells
                    
                    
                    

### Assessing performance of the new model

Now, as you did before, try to predict the performance (`root mean squared error`) of your model based on the `X_test`. If you obtain a strange error, look at the `hint` below...

In [ ]:
# ADD YOUR CODE HERE -- You can create new markdown and code cells
                    
                    
                    

<details><summary>🛟 Hint</summary>
    If you have not set your `OneHotEncoder` with the option `handle_unknown='ignore'`, you will get a bit fat error telling you: <code><strong style="color:red">ValueError</strong>: Found unknown categories ['Islamabad_Pakistan', 'Thousand Oaks_United States', 'Laohekou_China'] in column 0 during transform</code>. This means that three cities in your <code>X_test</code> where not present in your `X_train`, so are unknown.<br><br>
    This is normal and reflects real world data: sometimes, you cannot predict at <strong>training time</strong> all of the categories you will encounter at <strong>prediction time</strong>. If you set your `OneHotEncoder` as suggested above, it will ignore these categories and consider them as `nan`. Not ideal. There is a better way: read below to know more.</details>

# Features Hashing

`OneHotEncoding` is great because it gives us certainty of a 100% encoding without the risk of a clash of features; when using the `sparse_output` option, it can even handle some very large matrices fairly well, as we have seen with our `Feature Cross` before. But it has two major drawbacks:
1. `OneHotEncoding` forces you to encode using the same number of columns as there are categories - this can become problematic for datasets with very large cardinality (think for instance of encoding all the English words using `OneHotEncoding`: impractical!)
2. As you have discovered, if there is a feature category missing in the features you are trying to predict, `OneHotEncoding` cannot handle it and you need to set `handle_unknown=ignore`.

Instead, we will use a different encoding technique called `Feature Hashing`, also known as the ***hashing trick***. <a href="https://en.wikipedia.org/wiki/Feature_hashing">Wikipedia has a good page about this</a> in case you want to read more.

The idea is to encode the values dataset by binning our features into a define number of categories, and then transform this into a sparse vector by applying a deterministic `hashing algorithm`. The exact details of the algorithm are not essential here (see the link above for some of the math), what matters is the principle. Note too that you can use `Feature Hashing` to reduce the dimensionality of the encoded dataset if you decide to.

A few things to consider:

1. One immense benefit of this approach is that categories outside of our bins will not break the code (they will just be assigned to the wrong bin)
2. The number of bins (the `n_features` hyperparameter) is important. Too many of them, and you end up with the same cardinality as your original dataset (which is fine as long as the number of categories is not too high). Too few bins, and categories will start colliding, meaning you will introduce have more and more errors in your features.
3. For practical applications, it is recommended to keep the number of features in the `sklearn` class as its default, or if you change it, keep the number to a power of 2 (<a href="https://datascience.stackexchange.com/questions/77819/how-should-i-choose-n-features-in-featurehasher-in-sklearn">see this StackExchange post for more details</a>.
4. It is important to realise that when we use `Feature Hashing` we sacrifice potential accuracy in exchange for practicality: either because we have a feature with too large a cardinality, or because we cannot necessarily always be sure that the categories we have at training will contain all of the categories at prediction time. If neither of these conditions are true, you moght be better off sticking with `OneHotEncoding` for maximum precision.

Luckily for us, `sklearn` has a `FeatureHasher` class(<a href="https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.FeatureHasher.html">see the documentation</a>. This class does not need to be fitted (i.e. no need to call the `.fit()` method, though it exists): you can simply call the `transform()` method on the data as the hashing is deterministic and does not depend on the features.

Do the following:
1. Create a `ColumnTransformer` (let's call it `preprocessor` that contains your `numerical pipeline` from the previous step, and filters the same features as before (`['TempPreviousMonth','Latitude', 'Longitude']`)
2. Instead of the `categorical encoder` that contained a `OneHotEncoder`, create a different categorical pipeline that contains a `FeatureHasher` (part of the `feature_extraction` module of `sklearn`). **Important:** Set the `input_type` parameter of your `FeaterHasher` to `'string'` since this is what we are using for `City_Country`
3. If you run your pipeline with the `string` as it is, you will run into an issue. We need to provide a list of strings as input. The easiest is to quickly transform the `X_train.City_Country` column with a lambda function to be a list that contains the string (e.g. `'Kingston_Jamaica'` becomes `['Kingston_Jamaica']`). Make sure to also transform the `X_test.City_Country` for the test!
4. combine your `preprocessor` with a `LinearRegression()` model into a new `Pipeline` named `city_model`
5. Now `fit` the `city_model` to your `X_train` and `y_train`.


In [ ]:
# ADD YOUR CODE HERE -- You can create new markdown and code cells
                    
                    
                    

### Assessing performance of the model with `FeatureHasher`

Now, as you did before, try to predict the performance (`root mean squared error`) of your model based on the `X_test`, and this time save the results in a variable named `hash_score`. 

You should have no problem this time, and see a performance slightly better than your previous model (though not massively so...). 

In [ ]:
# ADD YOUR CODE HERE -- You can create new markdown and code cells
                    
                    
                    

### ☑️ Test your code

In [ ]:
from nbresult import ChallengeResult

result = ChallengeResult('final_score',
                         score = hash_score
)

result.write()
print(result.check())

# `CityEncoder` Transformer

Now you will write a complete Transfomer class for the `City` feature (do this in the `custom_transformers.py` file using `VSCode`). The transformer that will use the `City` and `Country` columns, combine them, and return a sparse matrix generated by the `FeatureHasher`. Use your new transformer to transform `X_train` and save the results under a variable named `test_cities`.

In [ ]:
from custom_transformers import CityEncoder

city_encoder = CityEncoder().fit(X_train, y_train)

test_cities = city_encoder.transform(X_train)

### ☑️ Test your code

In [ ]:
from nbresult import ChallengeResult

result = ChallengeResult('city_encoding',
                         sparsity = type(test_cities),
                         shape = test_cities.shape
)

result.write()
print(result.check())

# 🏁 Finished!

Well done! <span style="color:teal">**Push your exercise to GitHub**</span>, and move on to the next one.